<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>


## Mise a jour du catalog des notebooks et des READMEs
 - Génération du **catalog des notebooks** : [./logs/catalog.json](./logs/catalog.json)  
   Ce fichier comporte une liste détaillée de tous les notebooks et scripts.
   
   
 - Génération automatique de la table des matières et mise à jour des **README**
     - [README.md](../README.md)
     - [README.ipynb](../README.ipynb)



## Step 1 - Load modules and init

In [ ]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from IPython.display import display,Markdown
import sys
import datetime

sys.path.append('..')
import fidle.config as config
import fidle.cookindex as cookindex


## Step 2 - List of folders containing notebooks to be indexed :
Order wil be index order

In [ ]:
directories_to_index = {'LinearReg':'Linear and logistic regression', 
                        'IRIS':'Perceptron Model 1957', 
                        'BHPD':'Basic regression using DNN',
                        'MNIST':'Basic classification using a DNN',
                        'GTSRB':'Images classification with Convolutional Neural Networks (CNN)',
                        'IMDB':'Sentiment analysis with word embedding',
                        'SYNOP':'Time series with Recurrent Neural Network (RNN)',
                        'AE':'Unsupervised learning with an autoencoder neural network (AE)',
                        'VAE':'Generative network with Variational Autoencoder (VAE)',
                        'DCGAN':'Generative Adversarial Networks (GANs)',
                        'Misc':'Miscellaneous'
                        }

## Step 3 - Catalog of notebooks
### 3.1 - Build catalog

In [ ]:
cookindex.build_catalog(directories_to_index)
cookindex.build_default_profile()

### 3.3 Buil index

In [ ]:
catalog = cookindex. read_catalog()
styles = open('css/readme.css', "r").read()

lines_md=[]
lines_html=[styles]

for directory,title in directories_to_index.items():
    
    lines_md.append(f'\n### {title}')
    lines_html.append( f'<div class="fid_section">{title}</div>')
    
    entries = { k:v for k,v in catalog.items() if v['dirname']==directory }

    for id, about in entries.items():
        id          = about['id']
        dirname     = about['dirname']
        basename    = about['basename']
        title       = about['title']
        description = about['description']

        link=f'{dirname}/{basename}'.replace(' ','%20')
        md   = f'- **[{id}]({link})** - [{title}]({link})  \n'
        md  += f'{description}'
        html = f"""<div class="fid_line">
                       <span class="fid_id">
                           <a href="{link}">{id}</a>
                       </span> <a href="{link}">{title}</a><br>
                       <span class="fid_desc">{description}</span>
                  </div>
                """
        lines_md.append(md)
        lines_html.append(html)

index_md   = '\n'.join(lines_md)
index_html = '\n'.join(lines_html)

display(Markdown('**Index is :**'))

display(Markdown(index_md))
# display(HTML(index_html))


## Step 4 - Update README.md

In [ ]:
# ---- Load README.md
#
with open('../README.md','r') as fp:
    readme=fp.read()
    
# ---- Update index, version
#
readme = cookindex.tag('INDEX',   index_md,                readme)
readme = cookindex.tag('VERSION', f'**{config.VERSION}**', readme)

# ---- Save it
#
with open('../README.md','wt') as fp:
    fp.write(readme)

print('README.md is updated.')

## Step 5 - README.ipynb
Just execute README.ipynb

## Step 6 - More fun : Create and execute it :-)

Plus rigolo, on va fabriquer le README.ipynb et l'executer :-)

In [ ]:
# ---- Create Notebook from scratch
#
notebook = nbformat.v4.new_notebook()

# ---- Add a code cell
#
code = "from IPython.display import display,Markdown\n"
code+= "display(Markdown(open('README.md', 'r').read()))\n"
code+= "#\n"
code+= "# This README is visible under Jupiter LAb ! :-)"

new_cell = nbformat.v4.new_code_cell(source=code)
new_cell['metadata']= { "jupyter": { "source_hidden": True} }
notebook.cells.append(new_cell)

# --- Pour éviter une modification lors de l'ouverture du notebook
#     pas génante, mais nécessite de resauvegarder le document à la fermeture...
notebook['metadata']["kernelspec"] = {"display_name": "Python 3", "language": "python", "name": "python3" }

# ---- Run it
#
ep = ExecutePreprocessor(timeout=600, kernel_name="python3")
ep.preprocess(notebook,  {'metadata': {'path': '..'}})

# ---- Save it
#
with open('../README.ipynb', mode="w", encoding='utf-8') as fp:
    nbformat.write(notebook, fp)
print('README.ipynb built and saved')

In [ ]:
now = datetime.datetime.now()
print('Completed on : ', now.strftime("%A %d %B %Y, %H:%M:%S"))

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>